In [1]:
# Energy Consumption Patterns Model: Behavior and Technology
# ----------------------------------------------------------------------------------
# This model simulates household electricity demand under varying user behavior and
# technology configurations across an entire year (8760 hours). It loops through four
# defined scenarios (A–D), each with specific beta (behavioral multiplier) and tau
# (technology efficiency) parameters. The model uses Linopy to compute total electricity
# demand, cost, and CO₂ emissions.

import linopy as lp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------
# 1. SETUP SECTION
# ------------------------
# Create an 8760-hour datetime index for a full year (non-leap year)
time = pd.date_range("2024-01-01", periods=8760, freq="H")
end_uses = ["lighting", "water_heating"]

# Generate synthetic baseline profiles for full year based on 24-hour patterns
lighting_profile = [0.1]*6 + [0.3]*2 + [0.1]*8 + [0.4]*4 + [0.2]*4
water_profile = [0.0]*6 + [0.5]*2 + [0.1]*5 + [0.3]*2 + [0.6]*3 + [0.0]*6

# Repeat daily profiles for 365 days
days = 365
base_demand = pd.DataFrame({
    "lighting": lighting_profile * days,
    "water_heating": water_profile * days
}, index=time)

# Define behavior and tech efficiency parameters for all scenarios
scenario_params = {
    "A": {"beta": {"lighting": 1.2, "water_heating": 1.2}, "tau": {"lighting": 0.0, "water_heating": 0.0}},
    "B": {"beta": {"lighting": 0.8, "water_heating": 0.8}, "tau": {"lighting": 0.0, "water_heating": 0.0}},
    "C": {"beta": {"lighting": 1.2, "water_heating": 1.2}, "tau": {"lighting": 0.3, "water_heating": 0.2}},
    "D": {"beta": {"lighting": 0.8, "water_heating": 0.8}, "tau": {"lighting": 0.3, "water_heating": 0.2}},
}

# Flat rate price and CO₂ factor
price = pd.Series(0.30, index=time)  # €/kWh
co2_factor = 0.4  # kg CO₂/kWh

# ------------------------
# 2. SIMULATION ACROSS SCENARIOS
# ------------------------
summary = []
all_results = {}

for scenario in ["A", "B", "C", "D"]:
    print(f"\n--- Solving Scenario {scenario} ---")

    beta = scenario_params[scenario]["beta"]
    tau = scenario_params[scenario]["tau"]

    m = lp.Model(name=f"Scenario_{scenario}")

    x = {
        u: m.add_variables(name=f"x_{u}", coords={"time": time}, lower=0)
        for u in end_uses
    }

    x_total = m.add_variables(name="x_total", coords={"time": time}, lower=0)
    x_grid = m.add_variables(name="x_grid", coords={"time": time}, lower=0)

    for u in end_uses:
        adjusted = base_demand[u] * beta[u] * (1 - tau[u])
        m.add_constraints(x[u] == adjusted, name=f"demand_{u}")

    m.add_constraints(x_total == sum(x[u] for u in end_uses), name="total_demand")
    m.add_constraints(x_grid == x_total, name="grid_match")

    cost = (x_grid * price).sum()
    emissions = (x_grid * co2_factor).sum()
    m.add_objective(cost, sense="min")

    m.solve()

    cost_val = cost.solution.sum()
    demand_val = x_grid.solution.sum()
    emissions_val = emissions.solution.sum()

    summary.append({
        "Scenario": scenario,
        "Total Cost (EUR)": round(cost_val, 2),
        "Total Demand (kWh)": round(demand_val, 2),
        "Emissions (kg CO₂)": round(emissions_val, 2)
    })

    results = pd.DataFrame({
        "Lighting": x["lighting"].solution,
        "Water Heating": x["water_heating"].solution,
        "Total Demand": x_total.solution,
        "Grid Supply": x_grid.solution
    }, index=time)

    all_results[scenario] = results

# ------------------------
# 3. REPORTING SUMMARY
# ------------------------
summary_df = pd.DataFrame(summary).set_index("Scenario")
print("\nScenario Summary (Annual):\n")
print(summary_df)

# ------------------------
# 4. OPTIONAL PLOTTING (Daily average sample)
# ------------------------
sample_day = base_demand.index[:24]
fig, axs = plt.subplots(2, 2, figsize=(14, 8), sharex=True, sharey=True)
axs = axs.flatten()

for i, scenario in enumerate(["A", "B", "C", "D"]):
    results = all_results[scenario].loc[sample_day]
    ax = axs[i]
    results[["Lighting", "Water Heating"]].plot.area(ax=ax, stacked=True, alpha=0.6)
    ax.plot(results["Grid Supply"], linestyle="--", color="black", label="Grid Supply")
    ax.set_title(f"Scenario {scenario} (Day 1)")
    ax.set_ylabel("kWh")
    ax.grid(True)
    ax.legend()

plt.suptitle("Household Electricity Use by Scenario (First Day)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


C:\Users\linus\AppData\Local\Temp\ipykernel_16236\764262614.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time = pd.date_range("2024-01-01", periods=8760, freq="H")



--- Solving Scenario A ---


TypeError: Model.__init__() got an unexpected keyword argument 'name'